In [1]:
import os
import pandas as pd
import numpy as np

（1）先从http://www.kazusa.or.jp/codon 下载模式生物的密码子偏好统计数据；

（2）纠正格式。

In [9]:
!head -n 4 Compare_analysis/Arabidopsis_thaliana.txt

UUU 21.8(678320)  UCU 25.2(782818)  UAU 14.6(455089)  UGU 10.5(327640)
UUC 20.7(642407)  UCC 11.2(348173)  UAC 13.7(427132)  UGC  7.2(222769)
UUA 12.7(394867)  UCA 18.3(568570)  UAA  0.9( 29405)  UGA  1.2( 36260)
UUG 20.9(649150)  UCG  9.3(290158)  UAG  0.5( 16417)  UGG 12.5(388049)


In [21]:
model_org_dir = 'Compare_analysis'
my_org_dir = 'RSCU'

model_orgs = [os.path.join(model_org_dir, f) for f in os.listdir(model_org_dir) 
                  if f.endswith('.txt')]
my_orgs = [os.path.join(my_org_dir, f) for f in os.listdir(my_org_dir) 
                  if f.endswith('.txt')]

model_orgs

['Compare_analysis/Arabidopsis_thaliana.txt',
 'Compare_analysis/Populus_trichocarpa.txt',
 'Compare_analysis/Escherichia_coli.txt',
 'Compare_analysis/Saccharomyces_cerevisiae.txt']

In [50]:
import re

def parse_model_org(infile):
    """解析下载下来的模式物种数据
    """
    codons = []
    freq_kilo = []
    numbers = []
    with open(files[0]) as f:
        for line in f:
            if line.strip():
                line = [i.strip().replace('U', 'T') for i in re.split(r'\s+|\(|\)', 
                                                                      line.strip()) if i]
                for idx in range(0, len(line), 3):
                    codons.append(line[idx])
                    
                    freq_kilo.append(float(line[idx+1]))
                    numbers.append(int(line[idx+2]))
                    
    df = pd.DataFrame({'Codon': codons, 'freq_kilo': freq_kilo})
    
    return df


def parse_my_org(infile):
    """解析emboss上面计算得到的数据
    """
    df = pd.read_csv(infile, comment='#', header=0, delimiter=r"\s+")

    # 生成氨基酸简称的对应关系
    aa_tri = ['Gly', 'Ala', 'Val', 'Leu', 'Ile', 'Pro', 'Phe', 'Tyr', 'Trp', 'Ser', 
              'Thr', 'Cys', 'Met', 'Asn', 'Gln', 'Asp', 'Glu', 'Lys', 'Arg', 'His', 'Ter']
    aa_sing = ['G', 'A', 'V', 'L', 'I', 'P', 'F', 'Y', 'W', 'S', 'T', 'C', 'M', 'N', 'Q', 
               'D', 'E', 'K', 'R', 'H', '*']
    df_amino = pd.DataFrame({'Amino_acid':aa_tri, 'AA':aa_sing}).sort_values(by='AA')
    df_merge = df.merge(df_amino, on='AA', how='inner')
    
    return df_merge[['Amino_acid', 'Codon', 'Frequency']]

In [51]:
writer = pd.ExcelWriter('Step3_Compare_analysis.xlsx', engine='xlsxwriter')

for forg in my_orgs:
    my_species = forg.split('/')[-1].split('.')[0].split('_')[-1]
    df_org = parse_my_org(forg)
    for fm in model_orgs:
        model_species = fm.split('/')[-1].split('.')[0].split('_')[-1]
        name = my_species + '--' + model_species
        print(name)
        dfm = parse_model_org(fm)
        df = df_org.merge(dfm, on='Codon', how='inner')
        df['Ratio'] = df['Frequency']/df['freq_kilo']
        df[['Amino_acid', 'Codon', 'Ratio']].to_excel(writer, 
                                                      sheet_name=name, 
                                                      index=False)
writer.save()

floridulus--thaliana
floridulus--trichocarpa
floridulus--coli
floridulus--cerevisiae
giganteus--thaliana
giganteus--trichocarpa
giganteus--coli
giganteus--cerevisiae
sacchariflorus--thaliana
sacchariflorus--trichocarpa
sacchariflorus--coli
sacchariflorus--cerevisiae
sinensis--thaliana
sinensis--trichocarpa
sinensis--coli
sinensis--cerevisiae
transmorrisonensis--thaliana
transmorrisonensis--trichocarpa
transmorrisonensis--coli
transmorrisonensis--cerevisiae
spontaneum--thaliana
spontaneum--trichocarpa
spontaneum--coli
spontaneum--cerevisiae
bicolor--thaliana
bicolor--trichocarpa
bicolor--coli
bicolor--cerevisiae


In [52]:
len(set(df_org['Amino_acid']))

21

In [53]:
len(set(df['Amino_acid']))

21